<a href="https://colab.research.google.com/github/Malachyiii/Pawpularity-PytorchLightning-SwinComputerVision/blob/main/Pawpularity_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install timm fastai
!pip install -U fastai
!pip install wandb --quiet
!pip install pytorch-lightning

In [2]:
#Downloading the Data
!wget -O data.zip "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/25383/2684322/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1641577717&Signature=WkjwoZIt1aYIF42c%2Bit9JVun1Hmr6nIuk3RG%2Bmyzp1Y6i8ZRgD8t6TnzhVHr7VFtzMtrqqZSj1h75OjCCIJvJCUWz%2BK66PnCFa%2FGKvyql5X1NKBXgh%2FR%2FQCPasJmFPjgk0Nh3JXbtmYn%2BHMi7MN5xUEd%2FdSb2CYt7k7Arf%2BwZ9sAARTL4bmwkwVguNaQUNHBrXUsYqLsOX9ZsoL0sH05So3jP5m30AfjR9%2BmRP4KKS5Aw07N3Fy4hQxZjoQCdB2DPsqOSjwVqjwWlKeYEw3zB4X4cwfSxGcTl%2FIme5ofe3QbY4oEx7tJ2iwn1YL3i42q6Yrv3ZoPlaJnP%2B0wKH14Wg%3D%3D&response-content-disposition=attachment%3B+filename%3Dpetfinder-pawpularity-score.zip"

--2022-01-05 20:07:30--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/25383/2684322/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1641577717&Signature=WkjwoZIt1aYIF42c%2Bit9JVun1Hmr6nIuk3RG%2Bmyzp1Y6i8ZRgD8t6TnzhVHr7VFtzMtrqqZSj1h75OjCCIJvJCUWz%2BK66PnCFa%2FGKvyql5X1NKBXgh%2FR%2FQCPasJmFPjgk0Nh3JXbtmYn%2BHMi7MN5xUEd%2FdSb2CYt7k7Arf%2BwZ9sAARTL4bmwkwVguNaQUNHBrXUsYqLsOX9ZsoL0sH05So3jP5m30AfjR9%2BmRP4KKS5Aw07N3Fy4hQxZjoQCdB2DPsqOSjwVqjwWlKeYEw3zB4X4cwfSxGcTl%2FIme5ofe3QbY4oEx7tJ2iwn1YL3i42q6Yrv3ZoPlaJnP%2B0wKH14Wg%3D%3D&response-content-disposition=attachment%3B+filename%3Dpetfinder-pawpularity-score.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 64.233.189.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1030728211 (983M) [application/zip]
Saving to: ‘data.zip’

da

In [3]:
! unzip -qq "data.zip" -d data

replace data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import math
import fastai
from fastai.vision.all import *

import torch
from torch import nn
from torch.nn import functional as F

import torchvision
from torchvision import transforms as T
from torchvision.io import read_image

import timm
from timm import create_model

import sklearn
from sklearn.model_selection import StratifiedKFold

In [2]:
#timm.list_models(pretrained=True)

In [3]:
class args:
  folder_name = Path('../content/data/')
  seed = 1212
  num_splits = 10
  batch_size = 32
  num_workers = 8
  imagesize = 224
  model_name = 'tf_efficientnet_b8_ap'

In [4]:
df = pd.read_csv(args.folder_name/'train.csv')
df['filename'] = df['Id'].map(lambda x:str(args.folder_name/'train'/x)+'.jpg')

#feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'filename']]

In [5]:
df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,filename
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../content/data/train/0007de18844b0dbbb5e1f607da0606e0.jpg
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../content/data/train/0009c66b9439883ba2750fb825e1d7db.jpg
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../content/data/train/0013fd999caf9a3efe1352ca1b0d937e.jpg
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../content/data/train/0018df346ac9c1d8413cfcc888ca8246.jpg
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../content/data/train/001dc955e10590d3ca4673f034feeef2.jpg


In [6]:
seed = args.seed

device = torch.device("cuda:0")

set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [7]:
df = df.drop(columns=['Id'])
df = df.sample(frac=1).reset_index(drop=True)
df['norm_score'] = df['Pawpularity']/100


num_bins = int(np.ceil(2*((len(df))**(1./3))))

df['bins'] = pd.cut(df['norm_score'], bins=num_bins, labels=False)


df['fold'] = -1

strat_kfold = StratifiedKFold(n_splits=args.num_splits, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['bins'])):
    df.iloc[train_index, -1] = i

In [8]:
def rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(torch.sigmoid(input.flatten()), target))

In [9]:
def get_data(fold):

    df_f = df.copy()
    df_f['is_valid'] = (df_f['fold'] == fold)
    
    dls = ImageDataLoaders.from_df(df_f, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               valid_col='is_valid', #
                               seed=args.seed, #seed
                               fn_col='filename', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=args.batch_size, #pass in batch size
                               num_workers=args.num_workers,
                               item_tfms=Resize(args.imagesize), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), 
                                                          Saturation(), 
                                                          RandomErasing(p=.3, sh =.1, max_count = 2)])) #pass in batch_tfms
    
    return dls

In [10]:
#Valid Kfolder size
the_data = get_data(0)
assert (len(the_data.train) + len(the_data.valid)) == (len(df)//args.batch_size)

In [11]:
def get_learner(fold_num):
    data = get_data(fold_num)
    
    model = create_model(args.model_name, pretrained=True, num_classes=data.c)

    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=rmse).to_fp16()
    
    return learn

In [ ]:
for i in range(args.num_splits):

    print(f'Fold {i} results')
    
    learn = get_learner(fold_num=i)

    lr = learn.lr_find(end_lr=3e-2)

    learn.fit_one_cycle(5, lr.valley, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='rmse', comp=np.less, patience=2)]) 
    
    learn.recorder.plot_loss()

Fold 0 results


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b8_ap-00e169fa.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b8_ap-00e169fa.pth


epoch,train_loss,valid_loss,rmse,time


# **Reference**
###  [Petfinder Pawpularity EDA & fastai starter 🐱🐶](https://www.kaggle.com/tanlikesmath/petfinder-pawpularity-eda-fastai-starter)
###  [Petfinder& fastai with DataAugmentation KFold 10](https://www.kaggle.com/bobber/petfinder-fastai-with-dataaugmentation-kfold-10)
###  [Lovely Doggo with Bonky (fastai &timm)](https://www.kaggle.com/warotjanpinitrat/lovely-doggo-with-bonky-fastai-timm)